In [1]:
import django_jupyter
django_jupyter.init()

Notebook dir : D:\Django-AI-Agent\src\notebook
Django root  : D:\Django-AI-Agent\src
Django ready in Jupyter


In [2]:
from my_permit import permit_client as permit

In [11]:
from django.contrib.auth import get_user_model
from directories.models import Directory

User = get_user_model()

admin_users = User.objects.all()
directory = Directory.objects.all()

In [ ]:
from permit import PermitApiError

async def get_or_create_resource_instance(django_instance, resource_key="directory"):
    """
    Get or create a Permit resource instance for a Django model instance.

    Args:
        django_instance: The Django model instance.
        resource_key (str): The Permit resource key.

    Returns:
        dict: Permit resource instance data.
    """
    _key = f"{resource_key}:{django_instance.id}"

    try:
        # Try to get the resource instance
        _resource = await permit.api.resource_instances.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            # If not found, create the resource instance
            payload = {
                "key": f"{django_instance.id}",
                "resource": resource_key,
                "tenant": "default",
            }
            _resource = await permit.api.resource_instances.create(payload)
        else:
            raise e
    return _resource

In [ ]:
from permit import PermitApiError

async def get_or_create_resource_instance(django_instance, resource_key="directory"):
    """
    Get or create a Permit resource instance for a Django model instance.
    """
    _key = f"{resource_key}:{django_instance.id}"

    try:
        # ✅ CORRECT for your SDK: ONLY instance_key
        _resource = await permit.api.resource_instances.get(_key)

    except PermitApiError as e:
        if e.status_code == 404:
            # ✅ Create with SAME instance key
            payload = {
                "key": _key,
                "resource": resource_key,
                "tenant": "default",
            }
            _resource = await permit.api.resource_instances.create(payload)
        else:
            raise e

    return _resource


In [21]:
# from permit import PermitApiError

# async def get_or_create_resource_instance(django_instance, resource_key="directory"):
#     """
#     Get or create a Permit resource instance for a Django model instance.

#     Args:
#         django_instance: The Django model instance.
#         resource_key (str): The Permit resource key.

#     Returns:
#         dict: Permit resource instance data.
#     """
#     instance_key = f"{resource_key}:{django_instance.id}"

#     try:
#         # Try to get the resource instance
#         response = await permit.api.resource_instances.get(
#             resource=resource_key,
#             key=instance_key,
#             tenant="default"  # optional, include if you use tenants
#         )
#         return response

#     except PermitApiError as e:
#         if e.status_code == 404:
#             # If not found, create the resource instance
#             payload = {
#                 "key": instance_key,
#                 "resource": resource_key,
#                 "tenant": "default",  # optional
#                 "attributes": {
#                     "name": getattr(django_instance, "name", f"Instance {django_instance.id}")
#                 }
#             }
#             response = await permit.api.resource_instances.create(payload)
#             return response
#         else:
#             # Raise unexpected errors
#             raise e


In [25]:
async def assihn_user_ownership(django_user, django_instance, resource_key="directory"):
    """
    Assign ownership of a Permit resource instance to a Django user.
    Args:
        django_user: The Django user instance.
        django_instance: The Django model instance.
        resource_key (str): The Permit resource key.
    """
    _key = f"{resource_key}:{django_instance.id}"
    return await permit.api.users.assign_role({
        "user": f"{django_user.id}",
        "role": "admin",
        "resource_instance": _key,
        "tenant": "default"
    })


In [26]:
for obj in directory:
    response = await get_or_create_resource_instance(obj , resource_key="directory")
    role = await assihn_user_ownership(obj.owner, obj , resource_key="directory")
    print(response, role)

PermitValidationError: Validation error
path -> instance_id
	Resource instance ident must be in the format 'resource:key' (value_error)
